In [377]:
import pickle

In [378]:
with open("bbox.pkl", "rb") as f:
    bbox = pickle.load(f)

with open("mask.pkl", "rb") as f:
    mask = pickle.load(f)

In [379]:
bbox

[0.0, 0.0, 129.0, 479.0]

In [380]:
import albumentations as A
import cv2
import numpy as np
import random


def dilate_mask(mask, **kwargs):
    kernel_size = random.randrange(3, 21, 2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    noised_mask = cv2.dilate(mask, kernel)
    return noised_mask.astype(bool)


def erode_mask(mask, **kwargs):
    kernel_size = random.randrange(3, 21, 2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    noised_mask = cv2.erode(mask, kernel)
    return noised_mask.astype(bool)


# 原始图像和掩码
image = np.zeros(mask.shape, dtype=np.uint8)
mask = mask.astype(np.uint8)

# 定义变换
transform = A.Compose(
    [
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.5),
        A.OneOf([A.Lambda(mask=dilate_mask), A.Lambda(mask=erode_mask)], p=0.5),
    ]
)
# 对 mask 进行膨胀操作
# A.Lambda(mask=erode_mask),  # 对 mask 进行腐蚀操作


# 应用变换
transformed = transform(image=image, mask=mask)
transformed_image = transformed["image"]
transformed_mask = transformed["mask"]

In [381]:
BBOX_TRANSFORM = A.Compose(
    [A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.5, rotate_limit=10, p=0.5)],
    bbox_params=A.BboxParams(format="pascal_voc"),
)

In [382]:
def dilate_mask(mask, **kwargs):
    kernel_size = random.randrange(3, 21, 2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    noised_mask = cv2.dilate(mask, kernel)
    return noised_mask.astype(bool)


def erode_mask(mask, **kwargs):
    kernel_size = random.randrange(3, 21, 2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    noised_mask = cv2.erode(mask, kernel)
    return noised_mask.astype(bool)


MASK_TRANSFORM = transform = A.Compose(
    [
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.5),
        A.OneOf([A.Lambda(mask=dilate_mask), A.Lambda(mask=erode_mask)], p=0.5),
    ]
)


def add_noise_to_mask(mask: np.ndarray):
    image = np.zeros(mask.shape, dtype=np.uint8)
    mask = mask.astype(np.uint8)
    transformed = MASK_TRANSFORM(image=image, mask=mask)
    transformed_image = transformed["image"]
    transformed_mask = transformed["mask"]
    return transformed_mask.astype(bool)

In [383]:
type(bbox)

list

In [384]:
bbox.append("bbox")

In [385]:
transformed = BBOX_TRANSFORM(image=image, bboxes=[bbox])

In [386]:
transform_bbox = transformed['bboxes']

transform_bbox

[]

In [387]:
list(transform_bbox[0])[:-1]

IndexError: list index out of range